In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
from muon import prot as pt

from matplotlib import colors
%matplotlib inline

import matplotlib.pyplot as plt

import muon as mu

import scvi

## load datasets

In [ ]:
mdata_raw = mu.read("./citeseq_mdata_allsamples_filtered.h5mu")

In [ ]:
# load citeseq data
mdata = mu.read("./citeseq_mdata_allsamples_filtered_fine_clustering.h5mu")

In [ ]:
# load vento-tormo endometrial data
adata_endo = sc.read("./endometrium_all.h5ad")

In [ ]:
adata_raw = mdata_raw["rna"]

In [ ]:
mdata["rna"].obs['celltype_hires'] = mdata.obs['celltype_hires']

In [ ]:
# limit adata_raw.
adata = mdata["rna"]
adata_raw_filtered = adata_raw[adata_raw.obs_names.isin(adata.obs_names)].copy()
adata_raw_filtered

In [ ]:
del mdata
del mdata_raw

In [ ]:
adata_raw_filtered.obs = adata[adata_raw_filtered.obs_names].obs.copy()

In [ ]:
print(adata_raw_filtered.X)

In [ ]:
adata_raw_filtered.obs['cell_type'] = adata_raw_filtered.obs['celltype_hires'].copy()

In [ ]:
adata_raw_filtered.obs['celltype_hires']

In [ ]:
adata_raw_filtered.obs['cell_type']

In [ ]:
# Define file paths
matrix_path = "./quake_data/GSE111976_ct_endo_10x.mtx"
genes_path = "./quake_data/genes.tsv"
barcodes_path = "./quake_data/barcodes.tsv"
meta1_path = "./quake_data/GSE111976_summary_10x_day_donor_ctype.csv"
meta2_path = "./quake_data/GSE111976_summary_10x_donor_phase.csv"

# Load the sparse matrix
adata_quake = sc.read_mtx(matrix_path)

adata_quake = adata_quake.T

# Load genes and assign to adata.var_names
genes = pd.read_csv(genes_path, header=None, sep="\t")
adata_quake.var_names = genes[0].values
adata_quake.var['gene_id'] = genes[0].values

# Load barcodes and assign to adata.obs_names
barcodes = pd.read_csv(barcodes_path, header=None, sep="\t")
adata_quake.obs_names = barcodes[0].values

# Load metadata CSV files
meta1 = pd.read_csv(meta1_path)
meta2 = pd.read_csv(meta2_path)

meta1 = meta1.set_index('cell_name')

In [ ]:
adata_quake.obs = adata_quake.obs.join(meta1, how='left')
adata_quake.obs = pd.merge(adata_quake.obs, meta2, on="donor")
                           
print(adata_quake)

In [ ]:
adata_quake.obs['Stage'] = adata_quake.obs['phase_canonical']
adata_quake.obs['Stage'] = adata_quake.obs['Stage'].replace({'secretory_early': 'early-secretory',
                                                             'secretory_mid': 'mid-secretory',
                                                            'secretory_early-mid':'early-mid-secretory',
                                                            'secretory_late':'late-secretory'})

In [ ]:
adata_quake.write("quake_data.h5ad")

## preprocess both datasets

In [ ]:
adata_raw_filtered.raw = adata_raw_filtered
adata_quake.raw = adata_quake
adata_endo.X = adata_endo.raw.X
adata_endo.raw = adata_endo

## concatenate data

In [ ]:
adata_raw_filtered = adata_raw_filtered.copy()
adata_endo = adata_endo.copy()
adata_quake = adata_quake.copy()

In [ ]:
adata_endo.obs["batch_scvi"] = "roser"
adata_raw_filtered.obs["batch_scvi"] = "cite"
adata_quake.obs["batch_scvi"] = "quake"
adata_endo.obs["donor_id"] = adata_endo.obs["DonorID"]
adata_quake.obs["donor_id"] = adata_quake.obs["donor"]
adata_endo.obs["tissue_merged"] = adata_endo.obs["Stage"]
adata_raw_filtered.obs["tissue_merged"] = adata_raw_filtered.obs["tissue"]
adata_quake.obs["tissue_merged"] = adata_quake.obs["Stage"]

In [ ]:
adata_quake.obs = adata_quake.obs.set_index('Unnamed: 0')

In [ ]:
adata_raw_filtered.obs

In [ ]:
adata_both=adata_raw_filtered.concatenate(adata_endo,adata_quake, index_unique=None)

In [ ]:
adata_both.obs['tissue_merged']

In [ ]:
adata_both.layers["counts"] = adata_both.X.copy()  # preserve counts
sc.pp.normalize_total(adata_both)
sc.pp.log1p(adata_both)
adata_both.raw = adata_both  # freeze the state in `.raw`

In [ ]:
sc.pp.highly_variable_genes(adata_both, n_top_genes=5000, batch_key = 'batch_scvi', subset=True)

In [ ]:
adata_both.var

In [ ]:
#Non-harmonized data: 
sc.tl.pca(adata_both)
sc.pp.neighbors(adata_both, n_pcs=50, n_neighbors=30) 
sc.tl.umap(adata_both, min_dist=0.3)

In [ ]:
sc.pl.umap(adata_both, color=["batch_scvi",'tissue_merged'], ncols=1,
    frameon=False)

In [ ]:
adata_both.var['mt'] = adata_both.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata_both, qc_vars=['mt'], percent_top=None, log1p=True, inplace=True)

In [ ]:
adata_both.obs['donor_id'] = adata_both.obs['donor_id'].astype('category')

In [ ]:
sc.pl.violin(adata_both, keys='pct_counts_mt', groupby="donor_id", rotation=90)

In [ ]:
adata_scvi= adata_both.copy() 

#ADD DATA SOURCE AS CATEGORICAL_COVARIATE_KEYS
scvi.model.SCVI.setup_anndata(adata_scvi, layer="counts", categorical_covariate_keys=["donor_id",'tissue_merged'],
                              batch_key = "batch_scvi",
                       continuous_covariate_keys=["n_genes_by_counts", "total_counts", "pct_counts_mt"]
                              #,labels_key="celltype_wnn",
                             #unlabeled_category="Unknown"
                             )

In [ ]:
model = scvi.model.SCVI(adata_scvi, n_layers=2, n_latent=30, gene_likelihood="nb") 

In [ ]:
model

In [ ]:
model.train()

In [ ]:
model.save("all_cells_vento_quake_cite_concat_2025/")

In [ ]:
latent = model.get_latent_representation()

In [ ]:
adata_scvi.obsm["X_scVI"] = latent

In [ ]:
normalized_expr = model.get_normalized_expression(
    adata=adata_scvi,
    transform_batch=['cite', 'roser','quake']  
    ,library_size = 1e4)

In [ ]:
adata_scvi.layers['scvi_norm'] = normalized_expr.values

In [ ]:
sc.pp.log1p(adata_scvi, layer = 'scvi_norm')

In [ ]:
adata_scvi.layers['scvi_norm']

In [ ]:
# use scVI latent space for UMAP generation
sc.pp.neighbors(adata_scvi, use_rep="X_scVI")
sc.tl.umap(adata_scvi, min_dist=0.3)

In [ ]:
#harmonized data
sc.pl.umap(adata_scvi, color=["donor_id","batch_scvi","tissue_merged",
                             "PRL","IGFBP1","ARG1","HSD11B1"], ncols=1,size=10,
    frameon=False
           #,save='ROSER_stroma_decidualizationmarker.png'
           ,use_raw=False
           ,layer = 'scvi_norm'
          )

In [ ]:
batches = adata_scvi.obs['batch_scvi'].unique()

adata_both.X = adata_scvi.layers['counts'].copy()

normalized_subsets = []

# loop over each batch, normalize and log-transform separately
for batch in batches:
    print(f"Processing batch: {batch}")
    adata_subset = adata_scvi[adata_scvi.obs['batch_scvi'] == batch].copy()
    
    # normalize total counts for each cell in the subset
    sc.pp.normalize_total(adata_subset)
    
    # log-transform the data
    sc.pp.log1p(adata_subset)
    
    # Append the normalized subset to the list
    normalized_subsets.append(adata_subset)

# concatenate the normalized subsets back together
adata_norm = normalized_subsets[0].concatenate(*normalized_subsets[1:], batch_key='batch_scvi', index_unique=None)


In [ ]:
print(adata_norm.X)

In [ ]:
adata_norm.obs['donor_id'] = adata_norm.obs['donor_id'].astype('category')

In [ ]:
for col in adata_norm.obs.columns:
    if pd.api.types.is_categorical_dtype(adata_norm.obs[col]):
        adata_norm.obs[col] = adata_norm.obs[col].astype(str)

In [ ]:
adata_norm.write('all_endo_cite_norm.h5ad')

In [ ]:
adata_scvi.layers['norm_per_data_counts'] = adata_norm.X.copy()

In [ ]:
for col in adata_scvi.obs.columns:
    if pd.api.types.is_categorical_dtype(adata_scvi.obs[col]):
        adata_scvi.obs[col] = adata_scvi.obs[col].astype(str)

In [ ]:
adata_scvi.write('all_endo_cite_integrated_2025.h5ad')

In [ ]:
adata_scvi.obs['celltype_hires']

In [ ]:
# convert to categorical
adata_scvi.obs["celltype_hires"] = adata_scvi.obs["celltype_hires"].astype("category")
# add "NaN" as a valid category
adata_scvi.obs["celltype_hires"] = adata_scvi.obs["celltype_hires"].cat.add_categories("NaN")
# replace missing values with "NaN"
adata_scvi.obs["celltype_hires"].fillna("NaN", inplace=True)

In [ ]:
adata_scvi.obs.columns

In [ ]:
adata_scvi.obs['cell_type'].cat.categories

In [ ]:
adata_scvi.obs['Cell type'].cat.categories

In [ ]:
adata_scvi.obs['celltype_hires'].cat.categories

In [ ]:
# List the columns you want to merge
cols = ['Cell type', 'cell_type', 'celltype_hires']

# Replace string "nan" or "NaN" with np.nan for each column
for col in cols:
    if pd.api.types.is_categorical_dtype(adata_scvi.obs[col]):
        adata_scvi.obs[col] = adata_scvi.obs[col].astype(object)
    # Replace string 'nan' or 'NaN' with np.nan
    adata_scvi.obs[col] = adata_scvi.obs[col].replace({'nan': np.nan, 'NaN': np.nan})

In [ ]:
adata_scvi.obs['celltype_merged'] = (
    adata_scvi.obs['Cell type']
    .combine_first(adata_scvi.obs['cell_type'])
    .combine_first(adata_scvi.obs['celltype_hires'])
)

In [ ]:
adata_scvi.obs['celltype_merged']

In [ ]:
adata_fib = adata_scvi[adata_scvi.obs['celltype_merged'].isin(['eS','decFIB','dS','Stromal fibroblasts',
                                                              'hpFib'])]

## Figure 6F

In [ ]:
gene_prl = 'PRL'
gene_igfbp1 = 'IGFBP1'
cutoff_prl = 0
cutoff_igfbp1 = 0

if gene_prl in adata_fib.var_names and gene_igfbp1 in adata_fib.var_names:
    
    expr_prl = adata_fib[:, gene_prl].layers['norm_per_data_counts']
    if sparse.issparse(expr_prl):
        expr_prl = expr_prl.toarray().flatten()
    else:
        expr_prl = np.array(expr_prl).flatten()
    
    expr_igfbp1 = adata_fib[:, gene_igfbp1].layers['norm_per_data_counts']
    if sparse.issparse(expr_igfbp1):
        expr_igfbp1 = expr_igfbp1.toarray().flatten()
    else:
        expr_igfbp1 = np.array(expr_igfbp1).flatten()
    
    
    # create Boolean Masks for Positivity
    prl_positive = expr_prl > cutoff_prl
    igfbp1_positive = expr_igfbp1 > cutoff_igfbp1
    
    # Cells that are double-positive:
    double_positive = prl_positive & igfbp1_positive
    

    df = adata_fib.obs.copy()
    df['double_positive'] = double_positive
    
    # Count the number of double-positive cells per group.
    double_counts = df.groupby(['tissue_merged', 'batch_scvi'])['double_positive'].sum().unstack(fill_value=0)
    # Count the total cells per group.
    total_counts = df.groupby(['tissue_merged', 'batch_scvi']).size().unstack(fill_value=0)
    
    # Compute the frequency (fraction) of double-positive cells.
    frequency = double_counts / total_counts


    tissue_order = ['proliferative', 'early-secretory', 'early-mid-secretory', 'mid-secretory', 'late-secretory', 'parietalis', 'basalis']
    frequency = frequency.reindex(tissue_order, fill_value=0)
    
    batches = frequency.columns.tolist()  # e.g., ['cite', 'roser', 'quake']
    n_batches = len(batches)
    x = np.arange(len(tissue_order))
    width = 0.8 / n_batches  # width for each bar within a tissue group

    plt.figure(figsize=(6, 6))
    for i, batch in enumerate(batches):
        plt.bar(x + i * width, frequency[batch], width, label=batch)
        
    plt.xlabel('Tissue (tissue_merged)')
    plt.ylabel('Fraction of double-positive cells')
    plt.title('Fraction of cells positive for both PRL and IGFBP1 per tissue and dataset (batch_scvi)')
    plt.xticks(x + width * (n_batches - 1) / 2, tissue_order, rotation=45)
    plt.legend(title='Dataset (batch_scvi)')
    plt.tight_layout()
    plt.savefig('20250402fib_integr_freq_PRL_IGFBP1_dp.pdf', 
                  format='pdf', bbox_inches='tight')
    plt.show()
    
else:
    missing = [g for g in [gene_prl, gene_igfbp1] if g not in adata_fib.var_names]
    print(f"Gene(s) {missing} not found in the dataset.")


In [ ]:
frequency

In [ ]:
sc.pp.neighbors(adata_fib, use_rep="X_scVI")

In [ ]:
sc.tl.umap(adata_fib, min_dist=0.3)

In [ ]:
sc.tl.leiden(adata_fib, resolution = 0.5) 

In [ ]:
adata_fib.write('integr_adata_fib.h5ad')

In [ ]:
sc.pl.umap(adata_fib, color=["donor_id","batch_scvi","leiden","tissue_merged",'celltype_merged'], ncols=1, size=20,
    frameon=False)

In [ ]:
sc.pl.dotplot(adata_fib, groupby = 'leiden', var_names = ['PRL','IGFBP1','ARG1','CD82'
                                                         ,'IL1R2','IL1RL1',
                                                         'HSD11B1','VEGFA','IGF1','IGFBP5','ACTA2'],
              standard_scale='var',cmap = 'Blues',
             swap_axes=True)

In [ ]:
# Create a crosstab of leiden clusters (rows) by tissue_merged (columns)
ct = pd.crosstab(adata_fib.obs['tissue_merged'], adata_fib.obs['leiden'])

# Convert counts to frequencies by dividing each column by its total cell count.
freq = ct.div(ct.sum(axis=1), axis=0)

freq

## Figure 6H

In [ ]:
cluster_label = '12'

if cluster_label not in ct.columns:
    print(f"Cluster {cluster_label} not found in the data!")
else:
    # Calculate frequency: number of cells in cluster 20 divided by total cells per tissue.
    freq_cluster20 = ct[cluster_label] / ct.sum(axis=1)
    tissue_order = ['proliferative', 'early-secretory', 'early-mid-secretory', 'mid-secretory', 'late-secretory', 'parietalis', 'basalis']
    freq_cluster20 = freq_cluster20.reindex(tissue_order, fill_value=0)
    
    # Plot the frequency as a bar chart.
    plt.figure(figsize=(5, 5))
    freq_cluster20.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.xlabel("Tissue (tissue_merged)")
    plt.ylabel("Frequency of cluster 12")
    plt.title("Frequency of Cluster 12 per Tissue")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('20250402fib_integr_freq_cluster12.pdf', 
                  format='pdf', bbox_inches='tight')
    plt.show()

In [ ]:
freq_cluster20

In [ ]:
pd.crosstab(adata_fib.obs['tissue_merged'], adata_fib.obs['leiden'])

In [ ]:
sc.tl.dendrogram(adata_fib, groupby='tissue_merged')
sc.pl.dendrogram(adata_fib, groupby='tissue_merged')

In [ ]:
# Create a crosstab with tissues as rows and Leiden clusters as columns.
ct = pd.crosstab(adata_fib.obs['tissue_merged'], adata_fib.obs['leiden'])

# Normalize the counts per tissue (each row sums to 1).
ct_freq = ct.div(ct.sum(axis=1), axis=0)
tissue_order = ['proliferative', 'early-secretory', 'early-mid-secretory', 'mid-secretory', 'late-secretory', 'parietalis', 'basalis']
ct_freq = ct_freq.reindex(tissue_order, fill_value=0)
    
# Plot a stacked bar chart.
plt.figure(figsize=(10, 6))
ct_freq.plot(kind='bar', stacked=True, colormap='tab20', figsize=(10,6))
plt.ylabel('Frequency')
plt.title('Relative Frequency of Leiden Clusters per Tissue')
plt.legend(title='Leiden Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('20250402fib_integr_composition_leiden.pdf', 
                  format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Define your tissue order.
tissue_order = ['proliferative', 'early-secretory', 'early-mid-secretory',
                'mid-secretory', 'late-secretory', 'parietalis', 'basalis']

# Create a crosstab where columns are Leiden clusters and rows are tissues.
ct = pd.crosstab(adata_fib.obs['tissue_merged'], adata_fib.obs['leiden'])

# Normalize each cluster to sum to 1 (so for each cluster, we get the fraction contributed by each tissue).
ct_cluster_freq = ct.div(ct.sum(axis=0), axis=1)

cluster_annotation = {}
for cluster in ct_cluster_freq.columns:
    freq_series = ct_cluster_freq[cluster]
    # Get the top three tissues (by frequency) for this cluster.
    top_tissues = freq_series.sort_values(ascending=False).head(5) #was 3
    # Convert tissue names to their index in tissue_order.
    indices = [tissue_order.index(t) for t in top_tissues.index if t in tissue_order]
    weights = top_tissues[top_tissues.index.isin(tissue_order)].values
    if len(indices) > 0 and weights.sum() > 0:
        weighted_avg_index = sum(i * w for i, w in zip(indices, weights)) / sum(weights)
    else:
        weighted_avg_index = None
    cluster_annotation[cluster] = {'weighted_avg_index': weighted_avg_index, 
                                   'top_tissues': list(top_tissues.index)}

# Sort clusters by the weighted average index.
sorted_clusters = sorted(cluster_annotation.items(), key=lambda x: x[1]['weighted_avg_index'])

# Create new labels based on the rank and show the top three tissues.
new_labels_weighted = {cluster: f"{i+1}_{','.join(info['top_tissues'])}" 
                         for i, (cluster, info) in enumerate(sorted_clusters)}

print("New annotations based on weighted average of top three tissues:")
print(new_labels_weighted)

In [ ]:
cluster_annotation

In [ ]:
# Define the function to map weighted average index to a category.
def assign_cluster_category(weighted_avg_index):
    if weighted_avg_index < 1:
        return 'naive Fib'
    elif weighted_avg_index < 4:
        return 'hpFib1'
    elif weighted_avg_index < 5.5:
        return 'hpFib2'
    else:
        return 'decFib'


# Map each cluster to its new category label.
cluster_categories = {
    cluster: assign_cluster_category(info['weighted_avg_index'])
    for cluster, info in cluster_annotation.items()
}

print("Assigned cluster categories:")
print(cluster_categories)

In [ ]:
# Make sure that the leiden values in adata_fib.obs are strings (or convert the keys of new_labels_weighted accordingly)
adata_fib.obs['leiden_ordered'] = adata_fib.obs['leiden'].astype(str).map(cluster_categories)

# To check the assignment, you can view the first few rows:
print(adata_fib.obs[['leiden_ordered', 'leiden']].head())

## Figure 6G

In [ ]:
# Create a crosstab with tissues as rows and Leiden clusters as columns.
ct = pd.crosstab(adata_fib.obs['tissue_merged'], adata_fib.obs['leiden_ordered'])

# Normalize the counts per tissue (each row sums to 1).
ct_freq = ct.div(ct.sum(axis=1), axis=0)
tissue_order = ['proliferative', 'early-secretory', 'early-mid-secretory', 'mid-secretory', 'late-secretory', 'parietalis', 'basalis']
ct_freq = ct_freq.reindex(tissue_order, fill_value=0)
    
# Plot a stacked bar chart.
plt.figure(figsize=(10, 6))
ct_freq.plot(kind='bar', stacked=True, colormap='tab20', figsize=(10,6))
plt.ylabel('Frequency')
plt.title('Relative Frequency of Leiden Clusters per Tissue')
plt.legend(title='Leiden Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('20250403fib_integr_composition_leiden_ordered.pdf', 
                  format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
adata_fib.obs['leiden_ordered'] = adata_fib.obs['leiden_ordered'].astype('category')

sc.tl.dendrogram(adata_fib, groupby='leiden_ordered')
sc.pl.dendrogram(adata_fib, groupby='leiden_ordered')

## Figure S5H

In [ ]:
sc.pl.matrixplot(adata_fib, groupby = 'leiden_ordered', var_names = ['HOXA10','ESR1','PGR','GATA2','IGF1',
                                                                  'MT1H','MT1G','SGK1','PTN','HAND2','PTN',
                                                                  'C3','APOD','IFITM1','PLA2G2A','SERPINE1','IGFBP5','ACTA2','PRL','IGFBP1','ARG1','CD82'
                                                         ,'IL1R2','IL1RL1',
                                                         'HSD11B1','VEGFA'],
              standard_scale='var',cmap = 'Blues',
              save='final_matrixplot_13clusters_decidualization_20251012.pdf',
             swap_axes=False)